It may take a few seconds

In [ ]:
!wget -q --show-progress --progress=dot:giga https://www.dropbox.com/s/zza28peyjy8qgg6/T2tt_700_50.root
!wget -q --show-progress --progress=dot:giga http://www.hepforge.org/archive/cutlang/atla_opendata_had_ttbar.root

# CutLang parameters
- **file** = Root file name (**required**)
- **filetype** = Root file type (**required**) | ROOT file type can be:
 *LHCO*
 *FCC*
 *LVL0*
 *DELPHES*
 *ATLASVLL*
 *ATLMIN*
 *ATLASOD*
 *ATLASODR2*
 *CMSOD*
 *CMSODR2*
 *CMSNANO*
 *VLLBG3*
 *VLLSIGNAL*
- **file** : input root file
- **filetype** : input event format (do not change!)
- **adlfile** : the name we use for labeling the analysis 
- **events** : number of events used from each file
- **verbose** : frequency of processed event numbers written in output text
- **parallel** : enter 0 to speed up analysis with multiprocessing

# Output file
Output file name will be **histoOut-\<adlfile\>-\<file\>.root**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=example1 events=5000
#CLA ADL example1
info analysisExample
  title "Search for electroweak production of charginos and neutralinos in WH events in proton-proton collisions at sqrt(s) = 13TeV"
  experiment CMS
  id "SUS-16-043"
  publication "JHEP 11 (2017) 029"
  sqrtS 13.0
  lumi 35.9
  arXiv "1706.09933"
  hepdata " http://adress.org "
  doi "10.1007/JHEP11(2017)029"

region   testZ
  select      ALL              # to count all events
  select      Size (ELE) >= 2  # events with 2 or more electrons 
  histo       h1mReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1}m
  select      {ELE[0] ELE[1] }q == 0   # Z is neutral
  histo       h2mReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1}m
    

> You can run the same analysis with different root files (separating each with ;) and add different parameters (separating each with ;) for each (optional)

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=example2 events=5000
#CLA ADL example2

region   testZ
  select      ALL              # to count all events
  select      Size (ELE) >= 2  # events with 2 or more electrons 
  histo       h1mReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1}m
  select      {ELE[0] ELE[1] }q == 0   # Z is neutral
  histo       h2mReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1}m

region testdep
 testZ
 select      Size (JET) >= 1

region mapkap
 testdep
 select      Size (MUO) == 2


**Example 3** we learn...

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=example3 events=5000
#CLA ADL example3

define Zreco : MUO[0] MUO[1]

region   test
  select      ALL              # to count all events
  histo       hamet, "MET (GeV) ", 100, 0.0, 2000.0, MET
  select      Size (MUO) >= 2  # events with 2 or more electrons 
  histo       h1mReco, "Z candidate mass (GeV)", 100, 0, 200, {Zreco}m
  select      {Zreco}q == 0   # Z is neutral
  histo       h2mReco, "Z candidate mass (GeV)", 100, 0, 200, m(Zreco)
 
region mest
  test
  select size(JET) > 2


In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=example4 events=5000
#CLA ADL example4

define Zreco : ELE[0] ELE[1]

object goodEle using ELE
  select  Pt(ELE)        >   10
  select abs({ELE}Eta)   <   2.4
  select     {ELE}AbsEta ][  1.442 1.556

object BiJET : JET
  select bTag(JET) == 1

object myMu : MUO 
  select Pt(MUO) > 20

object myPho : PHO
  select Pt(PHO) > 10

object myTau : TAU 
  select Pt(TAU) > 10

define goodZreco : goodEle[0] goodEle[1]

region   test
  select      ALL              # to count all events
  select      Size(JET)     == 1  # events with 2 or more jets 
  select      Size(BJET)    == 1  # events with 2 or more jets 
  select      bTag(JET[0])  > 0
  select      bTag(JET[0])  == 1 
  select      Size(BiJET)   >= 1  # events with 2 or more jets 
  select      Size(ELE)     >= 2  # events with 2 or more electrons 
  select      Size(goodEle) >= 2  # events with 2 or more electrons 
  select      Size(MUO) >= 2  # events with 2 or more electrons 
  select      Size(myMu) >= 2  # events with 2 or more electrons 
  histo       h1mReco,     "Z candidate mass (GeV)", 100, 0, 200, {Zreco}m
  histo       h1mgoodReco, "Z candidate mass (GeV)", 100, 0, 200, {goodZreco}m
  select      {Zreco}q == 0       # Z is neutral
  select      {goodZreco}q == 0   # Z is neutral
  histo       h2mReco    , "Z candidate mass (GeV)", 100, 0, 200, m(Zreco)
  histo       h2mgoodReco, "Z candidate mass (GeV)", 100, 0, 200, m(goodZreco)


In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=example5 events=5000
#CLA ADL example5
#SkipHistos = 1 # to not print histos in efficiency table
#Skipeffs = 1   # to not print efficiencies at all

define Zreco : ELE[0] ELE[1]
define mZreco = m(Zreco)

object goodEle : ELE
  select   Pt(ELE)       >   10
  select abs({ELE}Eta )  <   2.4
  select     {ELE}AbsEta ][  1.442 1.556

define goodZreco : goodEle[0] goodEle[1]
define  wmc = 12.3 / 1.5

algo        preselection
  select      ALL              # to count all events
  select      Size(ELE)     >= 2  # events with 2 or more electrons 

algo        testg
  preselection
  weight      mcweight    wmc
  select      Size(goodEle) >= 2  # events with 2 or more electrons 
  histo       h1mgoodReco, "Z candidate mass (GeV)", 100, 0, 200, {goodZreco}m
  select      {goodZreco}q == 0   # Z is neutral
  histo       h2mgoodReco, "Z candidate mass (GeV)", 100, 0, 200, m(goodZreco)

algo        testA
  preselection
#  histo       h1mReco,     "Z candidate mass (GeV)", 100, 0, 200, {Zreco}m
  select      {Zreco}q == 0       # Z is neutral
  histo       h2mReco    , "Z candidate mass (GeV)", 100, 0, 200, mZreco


In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=example6 events=5000
#CLA ADL example6

define Zreco : ELE[0] ELE[1]

object goodEle : ELE
  select Pt(ELE)       >   10
  select   {ELE}AbsEta <   2.4
  select   {ELE}AbsEta ][  1.442 1.556

define goodZreco : goodEle[-1] goodEle[-1]

algo BestZ
  select      ALL                   # to count all events
  select      Size(ELE) >= 2        # events with 2 or more electrons 
  histo       hZRecoA, "Zee mass (GeV)", 100, 0, 200, m(Zreco)
  select      Size(goodEle) >= 2    # events with 2 or more good electrons 
  select      {goodZreco}m ~= 91.2  # find the pair yielding mass closest to Z    
  select      {goodZreco}q == 0     # Z is neutral
  histo       hZRecoB, "Zee mass best electrons (GeV)", 100, 0, 200, m(goodZreco)

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=example7 events=1000 verbose=100
#CLA ADL example7

define   WH1  : JET[-1] JET[-1] # - index means I do not know yet which jets to take
define   WH2  : JET[-3] JET[-3] # -3 here is different than -1, so these are two OTHER jets.
# --------------> -3 & -3 repeated indices in the same hadronic W
# --------------> this means the order is NOT important
# --------------> Jet1 + Jet2 = Jet2 + Jet1

###      chi2 for W finder
#define   Wchi2 : (( m(WH1 ) - 80.4)/2.1)^2 + (( m(WH2) - 80.4)/2.1)^2 
define   Wchi2 : (( {WH1}m - 80.4)/2.1)^2 + (( {WH2}m - 80.4)/2.1)^2 

## top quarks without b tagging 
define   Top1 : WH1 JET[-2] # take another jet
#----> 12 3 toplamakla, 13 2 toplamak ayni degil. cunku ilkinde bj candidate 3 ikincisinde 2 numarali jetler.
define   Top2 : WH2 JET[-4]
define   mTop1 : m(Top1)
define   mTop2 : m(Top2)
###      chi2 for top finder
#define   topchi2 : ((mTop1 - mTop2)/4.2)^2                  
define   totchi : ((mTop1 - mTop2)/4.2)^2  + Wchi2
#define   PTj1 : pT(WH1)
#define   totchi : (({WH1}m - 80.4)/2.1)^2 + (({WH2}m - 80.4)/2.1)^2 + Hstep(300.0 - PTj1)*(300.0/PTj1 - 1.0 )

algo besttop
  select   ALL                   # to count all events
  select   Size(JET) >= 6        # at least 6 jets
  histo    hdrjj  , "dT between jjs", 100, 0.0, 6.2, dR(JET_1, JET_0) 
#  select   MET < 100             # no large MET
 select   totchi  ~= 0  # find the tops and ws
 histo    hchi , "Histo of chi2", 50, 0, 500, totchi
 select   totchi < 10.5 # small than a certain value of chi2
 histo    hmWH1 , "Hadronic W reco (GeV)", 50, 50, 150, m(WH1)
 histo    hmWH2 , "Hadronic W reco (GeV)", 50, 50, 150, m(WH2)
 histo    hmTop1 , "Hadronic top reco (GeV)", 70, 0, 700, mTop1
 histo    hmTop2 , "Hadronic top reco (GeV)", 70, 0, 700, mTop2
 histo    hdrtt  , "dT between tops", 100, 0.0, 6.2, dR(Top1, Top2) 

**Union of two objects**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleUnions events=5000
#CLA ADL exampleUnions

object goodEle : ELE
  select  Pt(ELE)        >   10
  select abs({ELE}Eta)   <   2.4
  select     {ELE}AbsEta ][  1.442 1.556

object GMUO : MUO
  select  Pt(MUO)        >   10
  select abs({MUO}Eta)   <   2.4


object geps  : Union( MUO , ELE, TAU)            #add all leptons into a set
object gleps : Union( goodEle , GMUO )          #add all good electrons and good muons into another set

define Zreco = ELE[-1] ELE[-1]               #negative indices are to be determined by optimization functions  

region   test
  select      ALL              # to count all events
  select      Size (goodEle)  >= 1  # events with 2 or more electrons 
  select      Size(MUO) >= 1
  select      Size (GMUO)  >= 1  # events with 2 or more electrons 
  select      Size (gleps) > 2  # events with 2 or more leptons 
  select      Size(ELE)  >= 2  # events with 2 or more electrons 
  weight      mcw      1.1
  select      m(Zreco) ~= 90.1
   histo      hmz1, "before cut", 50, 0, 150, {Zreco}m
  select      m(Zreco) [] 70 110
   histo      hmz2, "after cut", 50, 60, 120, m(Zreco)
  select      {ELE_0 ELE_1 }q == 0   # Z is neutral

**Weights and Tables**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exTab events=5000
#CLA ADL example for tables and efficiencies

def effFunc = sqrt({ELE_0}pT)/({ELE_0}pT + {ELE_1}pT)

table   myWeight
        tabletype abuzittin
        nvars 1
        errors false
#       value   min    max  
         0.3    0.0   10.0
         0.5   10.0   20.0
         0.7   20.0   50.0
         0.8   50.0   70.0
         0.95  70.0   1000.0

table   my2dWeight
        tabletype abuzittin
        nvars 2
        errors false
#       value    min   max    min   max  
          0.1    0.0  10.0    -5.5  0.0
          0.1    0.0  10.0     0.0  5.5
          0.2   10.0  20.0    -5.5  0.0
          0.5   10.0  20.0     0.0  5.5
          0.4   20.0  50.0    -5.5  0.0
          0.4   20.0  50.0     0.0  5.5
          0.7   50.0  70.0    -5.5  0.0
          0.7   50.0  70.0     0.0  5.5
          0.95  70.0  1000.0  -5.5  0.0
          0.95  70.0  1000.0   0.0  5.5
    
table   myDet
        tabletype abuzittin
        nvars 2
        errors true
#       value   err-  err+      min    max        min    max   
          0.1   0.01  0.02       0.0   10.0      -5.5    0.0
          0.1   0.01  0.02       0.0   10.0       0.0    5.5
          0.2   0.01  0.03      10.0   20.0      -5.5    0.0
          0.2   0.01  0.03      10.0   20.0       0.0    5.5
          0.4   0.01  0.04      20.0   50.0      -5.5    0.0
          0.4   0.01  0.04      20.0   50.0       0.0    5.5
          0.7   0.01  0.05      50.0   70.0      -5.5    0.0
          0.7   0.01  0.05      50.0   70.0       0.0    5.5
          0.95  0.01  0.06      70.0   1000.0    -5.5    0.0
          0.95  0.01  0.06      70.0   1000.0     0.0    5.5

object myElectron
   take ELE
   select applyHM( myDet({ELE}pT , {ELE}Eta) == 1) # 0 to reject, 1 to accept.


region   test
  select  ALL             # to count all events
  select  Size(ELE) >= 1  # events generic electrons 
  select  Size(myElectron) >= 1 #events with detected electrons
  select  {ELE_0}Eta > 0
  weight  randWeight   1.123
  weight  ef1Weight myWeight( {ELE_0}pT )            # weight 1d
  weight  ef2Weight SUSEff( {ELE_0}pT, {ELE_0}Eta )  # weight 2d

  histo   h1ept, "E0 pt (GeV)", 100, 0, 2000, {ELE_0}pT

  select  applyHM( myWeight( {ELE}pT ) == 1) # 0 to reject, 1 to accept.    

**Sorting**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exSort events=5000
#CLA ADL Sorting

object goodEle : ELE
  select   Pt(ELE_)       >   10
  select abs({ELE_}Eta )  <   2.4
  select     {ELE_}AbsEta ][  1.442 1.556


algo Dsorting
  select      ALL                   # to count all events
  select      Size(goodEle) >= 2 
  Sort        {goodEle_ }E descend
  histo       h1d, "test electron E sorting Des", 100, 0, 1000, {goodEle_0}E 
  Sort        {goodEle_ }E ascend
  histo       h1a, "test electron E sorting Asc", 100, 0, 1000, {goodEle_0}E 
  Sort        {goodEle_ }Pz descend
  histo       h2d, "test electron Pz sorting Des", 100, 0, 1000, {goodEle_0}Pz 
  Sort        {goodEle_ }Pz ascend
  histo       h2a, "test electron Pz sorting Asc", 100, 0, 1000, {goodEle_0}Pz 


**Saving**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleSave events=5000
#CLA ADL saving example


region   test
  select  ALL              # to count all events
  select  Size (ELE) >= 2  # events with 2 or more electrons 
#  weight   randWeight   1.123
  save     mid1file
# reject   Size (ELE) > 4  # events with 2 or more electrons 
#  histo    h1mReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1}m
# histo    h1mEta, "Z candidate Eta", 100, -5, 5, {ELE_0 ELE_1}Eta
  save     end1file

# region mest
#  select  ALL
#  select  Size (MUO) >= 2 
#  select  Size (ELE) >= 2  # events with 2 or more electrons 
#  save    mid2file
#  weight  bWeight   2.321
#  Save    end2file


**Histograms**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleHistos events=5000
# Object selection
object jets
  take jet
  select pT(jet) > 30
  select abs(eta(jet)) < 2.4

histoList jetHistos
  histo hjet1pt , "jet 1 pT (GeV)", 40, 0, 1000, pT(jets[0])
  histo hjet2pt , "jet 2 pT (GeV)", 40, 0, 1000, pT(jets[1])
  histo hjet3pt , "jet 3 pT (GeV)", 40, 0, 1000, pT(jets[2])
  histo hjet4pt , "jet 4 pT (GeV)", 40, 0, 1000, pT(jets[3])
  histo hmetjet1pt, "MET vs. jet pT", 20, 300, 1300, 20, 0, 1000, MET, pT(jets[0])


# Event selection
region presel
  select ALL
  select size(jets) >= 4
  histo hmetjet1pt, "MET vs. jet 1 pT", 20, 300, 1300, 20, 0, 1000, MET, pT(jets[0])
  select MET > 300
  jetHistos 


**Binning**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleBin events=5000
#CLA ADL example on bins

object jets : JET
 select Pt(JET) >  10
 histo    h1jetpt, "Jet PT (GeV)", 100, 0, 200, PT(JET)

define EHT = pT(ELE_0) +  pT(ELE_1)
define MHT = pT(MUO_0) +  pT(MUO_1)
define  HT = fHT(jets)

region   test
  select  ALL              # to count all events
  select  Size(JET) >= 2
  select  Size(jets) >= 2
  select  Size (ELE) >= 2  # events with 2 or more electrons 
  histo    h1eReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1 }m
  histo    h1eEta, "Z candidate Eta", 100, -5, 5, { ELE_0 ELE_1 }Eta
#  save     testfile
  Bin EHT [] 0.0 300 && HT [] 000  500  # 0
  Bin EHT [] 0.0 300 && HT [] 500 1000  # 1
  Bin EHT [] 0.0 300 && HT      > 1000  # 2
  Bin EHT [] 300 500 && HT [] 000  500
  Bin EHT [] 300 500 && HT [] 500 1000  # 4
  Bin EHT [] 300 500 && HT      > 1000
  Bin EHT [] 500 750 && HT [] 000 1000  # 6
  Bin EHT [] 500 750 && HT      > 1000
  Bin EHT      > 750 && HT [] 750 1500  # 8
  Bin EHT      > 750 && HT      > 1500  # 9

region   mest
  select  Size (MUO) >= 2  # events with 2 or more electrons 
  histo   h1mReco, "Z candidate mass (GeV)", 100, 0, 200, {MUO_0 MUO_1}m
  select  MHT <1300 && HT < 1500
  bin MHT 300 500 700 1000


**Counting**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleCount events=5000
#CLA ADL counting example

countsformat results
  process est, "Total estimated BG", stat, syst 
  process obs, "Observed data"

countsformat bgests
  process lostlep, "Lost lepton background", stat, syst
  process zinv, "Z --> vv background", stat, syst
  process qcd, "QCD background", stat, syst

region   presel
  select      ALL              # to count all events
  select      Size (ELE) >= 2  # events with 2 or more electrons 
  histo       h1mReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1}m
#  select      {ELE[0] ELE[1] }q == 0   # Z is neutral
#  histo       h2mReco, "Z candidate mass (GeV)", 100, 0, 200, {ELE_0 ELE_1}m
  bin Size(ELE) == 2
  counts results 230.0 + 16.0 - 10.0 + 10.0 - 12.0  , 224.0
  counts bgests 105.0 +16.0 - 10.0 +-1.0 , 123.0 +-2.0 +-12.0 , 2.3 +-0.5 +-1.4 

#region searchbins
#  presel
#  # Table 3, 1-10
#  bin MHT [] 300 350 and HT [] 300 600 and size(jets) [] 2 3 and size(bjets) == 0
#  counts bgests 38870 +- 320 +- 580 , 89100 +- 200 +- 2600 , 1800 +- 1000 +- 1200
#  counts results 129800 +- 1100 +- 2800 , 130718
#  bin MHT [] 300 350 and HT [] 600 1200 and size(jets) [] 2 3 and size(bjets) == 0
#  counts bgests 2760 +- 61 +- 39 , 4970 +- 50 +- 150 , 330 +- 180 +- 160
#  counts results 8060 +- 200 +- 220 , 7820
#  bin MHT [] 300 350 and HT > 1200 and size(jets) [] 2 3 and size(bjets) == 0
#  counts bgests 181 +- 17 +- 3 , 308 +- 12 +- 18 , 62 +- 34 +- 27
#  counts results 552 +- 40 +- 32 , 514


**Jet constituents**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleConstituents events=5000
#CLA ADL jet constituents example

object goodJet using JET
  select q(JET constituents ) == 0
# select q(JET:constituents ) == 0  
# select q(constituents(Jet) ) == 0  
  select pT( JET ) > 50
  select Sum(pT(JET constituents ) ) < 40  # PT from remaining constituents
#        ^^^^^^^^^^^^^^\_this is a cut on one JET. loop is over the constituents
#  select VertexTR(Jet constituents) < 4   # remove if far PrimaryVertex'ed constituents


region   test
  select  ALL              # to count all events
  select  Size (goodJet) >= 5  # events with 2 or more electrons 
##select Sum(pT(JET constituents ) ) < 40  # PT from remaining constituents
##       ^^^^^^^ such a cut doesnt make sense here, this is event loop
# select Sum(pT(goodJet)) > 55
#        ^^^^^^^\_this is a loop over all jets, so it is OK.

**Using loops and reducers**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleLoop events=5000
# ADL example for loops and reducers

object goodjet : JET
     select      pT(JET)  > 20
     select abs(eta(JET)) < 2.5
#    select Sum( pT(JET)) >= 20 #### This doesnt make sense. 
# in a new object, The loop is over individual jets, not over all event variables


region   signal
  select  ALL              # to count all events
  select  Size( JET ) > 2
  select  Sum ( pT(JET) ) >= 20
  select  Size( ELE ) > 0
  select  min ( dR(JET[0:2], ELE[0] )) > 0.9
  select  min ( dR(JET[0: ], ELE[0] )) > 0.9
  select  min ( dR(ELE[0]  , JET    )) > 1.2

 region   deneme
   select ALL
   select Size(ELE) > 0
   select Size(goodjet) > 2
   select min ( dR(goodjet[0: ], ELE[0] )) > 0.9
   select min ( dR(goodjet[0:2], ELE[0] )) > 1.3
#  select Size(MUO) > 2
#  select min ( dR(MUO[0:2], ELE[0] )) > 0.9


**Generator Level particles**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleGen events=5000
object genelectrons : GEN
 select {GEN}PDGID == -11 
 select {GEN}pT > 5
 select {GEN}AbsEta < 2.4

object genmuons : GEN
 select PDGID(GEN) == -13
 select Pt(GEN) > 5
 select Abs( Eta(GEN) ) < 2.4

region test1
  select Size(genelectrons) == 1
  select vertexY(genelectrons_0) > 0

#region test2
# select {GEN_1}PDGID == -11
# select {GEN_1}pT > 5
# select {GEN_1}AbsEta < 2.4
# select vertexX(GEN_1) > 0

**Using external functions**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleExtern events=5000
#CLA ADL example for external functions

#object megajets
#  take JET
#  select fmegajets(JET) == 2

#object hemisjets
#  take JET
#  select fhemisphere(JET, 2, 3) == 2

region   test
  select  ALL              # to count all events
  select  Size (JET) >= 2  # events with 2 or more electrons 
#  select fMT2(hemisjets[0], hemisjets[1], METLV[0]) > 40.0
  select  fMT2(JET[0], JET[1], METLV[0]) > 40.0
  select  fMT2(JET[0], JET[1], METLV[0]) > 45.0
  select  fMT2(JET[0], JET[1], METLV[0]) > 50.0


#  select fMTauTau(ELE[0], ELE[1], METLV[0]) [] 0 160

**Combinations for partitioning**

In [ ]:
%%cutlang file=T2tt_700_50.root filetype=DELPHES adlfile=exampleComb events=5000
object jets : JET
   select  {JET}Pt > 10
   select  {JET}Eta < 4

###### 12 21 13 31
object hZs : COMB( jets[-1] jets[-2] ) alias ahz
     select { ahz }AbsEta < 3.0
     select {jets[-2] }Pt > 2.1
     select {jets[-1] }Pt > 5.1
     select {jets[-1], ahz }dR < 0.6   #--- means a member of hZs and its constituent number 1, not a loop between all members and all constituents
     select { ahz }m [] 10 200           # does get the paricle numbers (2) fromt the last cut

define zz : hZs[-1]
define zhad : jets[-1] jets[-2]
define zham : {hZs[-1]}m
define zhbm : {hZs[-2]}m
#define chi2 : (zham - zhbm)^2
define chi2 : (zham - 91.2)^2 + (zhbm - 91.2)^2

object goodeles : ELE
   select  {ELE}Pt > 10
   select  {ELE}AbsEta < 4

region testA
        select ALL                        # to count all events
# count number size are all the same.
        select Size(jets) > 2             # we need at least 2 jets to reconstruct Zhads
        select Count(hZs)  >= 2            # we need at least 2 Zhad candidates.
# 4Jetli bir olayim var. 12, 21, 13, 31, 23, 32, 14, 41, 24, 42, 34, 43
#                        ^^------------------------------------- ^^  ^^
        select { zz }m ~= 91.2            # we kill here overlapping candidates. 
        select chi2  ~= 0            # we kill here overlapping candidates. 
        histo  hchi2, "chi2 ", 50, 0, 500, chi2
        histo  bm1hZ, "mass Hrec GeV", 50, 50, 150, zham
        histo  bm2hZ, "mass Hrec GeV", 50, 50, 150, zhbm
        histo  bm12hZ,"mass 1 vs 2 GeV", 50, 50, 150, 50, 50, 150, zham, zhbm
        select chi2  <= 100            # we kill bad candidates. 
        histo  am1hZ, "mass Hrec GeV", 50, 50, 150, zham
        histo  am2hZ, "mass Hrec GeV", 50, 50, 150, zhbm
        histo  am12hZ,"mass 1 vs 2 GeV", 50, 50, 150, 50, 50, 150, zham, zhbm

